In [1]:
from qtn.bimax import BiMax
from qtn.new_bimax import new_BiMax
from qtn.util import f1, j0
from qtn.bimax_util import z_b, j02
import numpy as np
from sympy.mpmath import mp,fp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ant_len = 50      # m (monopole) 
ant_rad = 1.9e-4  # m
base_cap = 20e-12 # Farad
al_ratio = ant_rad / ant_len
print(al_ratio)

3.8e-06


In [3]:
p1 = BiMax(ant_len, ant_rad, base_cap)
p2 = new_BiMax(ant_len, ant_rad, base_cap)

In [24]:
wrel, l, n, t, tc = 1.1, 5, 0.2, 5, 1
wc = wrel * fp.sqrt(1 + n)

In [25]:
mp.dps = 25
print(p1.za_l(wc, l, n, t, tc))
mp.dps = 15

(1802323.641556297364385287 + 16619100.48583821462121116j)


In [26]:
# mp.dps = 55
# print(p1.za_l(wc, l, n, t, tc))
# mp.dps = 15

In [27]:
mp.dps = 15
p2.za_l(wrel, l, n, t, tc)

guess =  2.645105682349282
z0 =  2.92318658905243
dl_imag =  0.20936405902103233
direct evaluating integral when peak is small


mpc(real='1802323.6044319759', imag='16614948.600398194')

In [33]:
class random_num:
    def __init__(self):
        self.num = np.random.rand()

In [34]:
f = random_num()

In [35]:
for _ in range(3):
    print(f.num)

0.0601900708108255
0.0601900708108255
0.0601900708108255
